# Sentiment Analysis and Extraction of Tweets 
## CS156 Final Project
## Bella Buchanan


In this project, I create a submission for Kaggle's Tweet Sentiment Extraction competition. I thought this would be a fun project and it is also motivating to know that others are trying to tackle this relevant problem. With the 2020 elections coming up, sentiment analysis of social media is an especially pressing issue.

The prompt is not merely to extract the sentiment from a tweet, but to extract the specific words in the tweet that reflect that sentiment. 

To extend this project, I have decided to first build models for **Sentiment Analysis** and then proceed to the competition task of **Sentiment Extraction** 

In [0]:
import numpy as np 
import pandas as pd 
import random
random.seed(30)

import json

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Data loading and pre-processing 

In [89]:
train_data = pd.read_csv("/content/drive/My Drive/train.csv")
test_data = pd.read_csv("/content/drive/My Drive/test.csv")
print(train_data.shape)
print(test_data.shape)

(27481, 4)
(3534, 3)


In [90]:
#remove blank entries
train_data.dropna(inplace = True)
test_data.dropna(inplace = True)
print(train_data.shape)
print(test_data.shape)

(27480, 4)
(3534, 3)


In [0]:
#Take a small subset of the data to tune parameters
tuning_data = train_data.sample(frac=0.10,random_state=200)
train_data = train_data.drop(tuning_data.index)

#We need a test set with correct answers so that we can evaluate it
#The competition is still ongoing so we do not have the correct answers
test_data2 = train_data.sample(frac=0.15,random_state=200)
train_data = train_data.drop(test_data2.index)


In [92]:
print(train_data.shape)

(21022, 4)


In [93]:
train_data.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
5,28b57f3990,http://www.dothebouncy.com/smf - some shameles...,http://www.dothebouncy.com/smf - some shameles...,neutral


In [0]:
#conver to np arrays 
train = np.array(train_data)
test = np.array(test_data)
tune = np.array(tuning_data)
test2 = np.array(test_data2)

!mkdir -p data

"""
SETTINGS
"""

use_cuda = True # whether to use GPU or not

In [0]:

"""
These functions put the data in the format:
Question: sentiment
Context: text
Answer: selected text
"""

# Adpated from https://www.kaggle.com/cheongwoongkang/roberta-baseline-starter-simple-postprocessing
def find_all(input_str, search_str):
    l1 = []
    length = len(input_str)
    index = 0
    while index < length:
        i = input_str.find(search_str, index)
        if i == -1:
            return l1
        l1.append(i)
        index = i + 1
    return l1

def do_qa_train(train):

    output = []
    for line in train:
        context = line[1]

        qas = []
        question = line[-1]
        qid = line[0]
        answers = []
        answer = line[2]
        
        answer_starts = find_all(context, answer)
        for answer_start in answer_starts:
            answers.append({'answer_start': answer_start, 'text': answer.lower()})
            break
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})

        output.append({'context': context.lower(), 'qas': qas})
        
    return output

qa_train = do_qa_train(train)

with open('data/train.json', 'w') as outfile:
    json.dump(qa_train, outfile)

qa_tune = do_qa_train(tune)

with open('data/tune.json', 'w') as outfile:
    json.dump(qa_tune, outfile)

qa_test2 = do_qa_train(test2)

with open('data/test2.json', 'w') as outfile:
    json.dump(qa_test2, outfile)

In [96]:
%%time

"""
Prepare testing data in QA-compatible format
"""

def do_qa_test(test):
    output = []
    for line in test:
        context = line[1]
        qas = []
        question = line[-1]
        qid = line[0]
        
        answers = []
        answers.append({'answer_start': 1000000, 'text': '__None__'})
        qas.append({'question': question, 'id': qid, 'is_impossible': False, 'answers': answers})
        output.append({'context': context.lower(), 'qas': qas})
    return output

qa_test = do_qa_test(test)

with open('data/test.json', 'w') as outfile:
    json.dump(qa_test, outfile)

CPU times: user 77 ms, sys: 2 ms, total: 79 ms
Wall time: 80.8 ms


In [97]:
%%time
!pip install simpletransformers
from simpletransformers.question_answering import QuestionAnsweringModel

CPU times: user 14.1 ms, sys: 30.8 ms, total: 44.9 ms
Wall time: 3.44 s


In [0]:
MODEL_PATH = '/content/drive/My Drive/distilbert-base-uncased-distilled-squad/'

## Tuning Parameters
We adjust parameters on the tuning dataset to choose the best ones 

In [0]:
parameters = {
    "1": {
                   "reprocess_input_data": True,
                   "overwrite_output_dir": True,
                   "learning_rate": 4e-05, #default
                   "num_train_epochs": 1,
                   "adam_epsilon": 1e-08, #default
                   "max_seq_length": 192,
                   "weight_decay": 0, #default
                   "doc_stride": 64,
                   "save_model_every_epoch": False,
                   "fp16": False,
                   "do_lower_case": True,
                   "warmup_steps": 200,
                   "warmup_ratio": 0.06
                },
    "2": {
                   "reprocess_input_data": True,
                   "overwrite_output_dir": True,
                   "learning_rate": 8e-05, #changed
                   "num_train_epochs": 1,
                   "adam_epsilon": 1e-08, 
                   "max_seq_length": 192,
                   "weight_decay": 0.001, #changed 
                   "doc_stride": 64,
                   "save_model_every_epoch": False,
                   "fp16": False,
                   "do_lower_case": True,
                    "warmup_steps": 200,
                    "warmup_ratio": 0.06 
                },
    "3": {
                   "reprocess_input_data": True,
                   "overwrite_output_dir": True,
                   "learning_rate": 8e-05, #changed
                   "num_train_epochs": 1,
                   "adam_epsilon": 1e-08,
                   "max_seq_length": 192,
                   "weight_decay": 0.1, #changed
                   "doc_stride": 64,
                   "save_model_every_epoch": False,
                   "fp16": False,
                   "do_lower_case": True,
                    "warmup_steps": 200,
                    "warmup_ratio": 0.06
                },
    "4": {
                   "reprocess_input_data": True,
                   "overwrite_output_dir": True,
                   "learning_rate": 8e-04, #changed
                   "num_train_epochs": 1,
                   "adam_epsilon": 1e-08,
                   "max_seq_length": 192,
                   "weight_decay": 0.001, #changed
                   "doc_stride": 64, 
                   "save_model_every_epoch": False,
                   "fp16": False,
                   "do_lower_case": True,
                    "warmup_steps": 200,
                    "warmup_ratio": 0.06
                }
    }

In [102]:
#recording the performing with different parameters
results1 = []
for key, param in parameters.items():
    
    model = QuestionAnsweringModel('distilbert', 
                               MODEL_PATH, 
                              args=param,
                              use_cuda=use_cuda)

    model.train_model('data/tune.json')
    result1, text = model.eval_model('data/tune.json')
    results1.append(result1)


100%|██████████| 2748/2748 [00:03<00:00, 740.01it/s]


Running loss: 3.469150

 80%|████████  | 16917/21022 [00:40<00:05, 763.96it/s]

Running loss: 2.910099

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


...

Running loss: 0.930259




100%|██████████| 2748/2748 [00:03<00:00, 824.08it/s]



100%|██████████| 2748/2748 [00:04<00:00, 668.91it/s]


Running loss: 3.597008




100%|██████████| 2748/2748 [00:03<00:00, 833.20it/s]


In [107]:
#the parameters for the 3rd model were best, proceed with those 
results1

[{'correct': 1330,
  'eval_loss': -5.210076491500056,
  'incorrect': 499,
  'similar': 919},
 {'correct': 1361,
  'eval_loss': -5.428132014219151,
  'incorrect': 484,
  'similar': 903},
 {'correct': 1351,
  'eval_loss': -5.881345352461172,
  'incorrect': 495,
  'similar': 902},
 {'correct': 217,
  'eval_loss': -3.0898431026658346,
  'incorrect': 826,
  'similar': 1705}]

## The Model 

In [110]:
%%time
model = QuestionAnsweringModel('distilbert', 
                               MODEL_PATH, 
                               args={"reprocess_input_data": True,
                                "overwrite_output_dir": True,
                                "learning_rate": 8e-05, #changed
                                "num_train_epochs": 5,
                                "adam_epsilon": 1e-08,
                                "max_seq_length": 192,
                                "weight_decay": 0.001, #changed
                                "doc_stride": 64,
                                "save_model_every_epoch": False,
                                "fp16": False,
                                "do_lower_case": True,
                                  "warmup_steps": 200,
                                  "warmup_ratio": 0.06
                                    },
                              use_cuda=True)

CPU times: user 1.16 s, sys: 179 ms, total: 1.34 s
Wall time: 1.5 s


In [111]:
#training!
model.train_model('data/train.json')



  0%|          | 0/21022 [00:00<?, ?it/s]

  0%|          | 79/21022 [00:00<00:26, 783.02it/s]

  1%|          | 156/21022 [00:00<00:26, 775.43it/s]

  1%|          | 227/21022 [00:00<00:27, 754.28it/s]

  1%|▏         | 307/21022 [00:00<00:27, 765.46it/s]

  2%|▏         | 376/21022 [00:00<00:27, 741.04it/s]

  2%|▏         | 448/21022 [00:00<00:28, 731.16it/s]

  2%|▏         | 525/21022 [00:00<00:27, 741.43it/s]

  3%|▎         | 602/21022 [00:00<00:27, 749.26it/s]

  3%|▎         | 678/21022 [00:00<00:27, 748.33it/s]

  4%|▎         | 750/21022 [00:01<00:28, 720.66it/s]

  4%|▍         | 821/21022 [00:01<00:28, 714.85it/s]

  4%|▍         | 897/21022 [00:01<00:27, 727.34it/s]

  5%|▍         | 969/21022 [00:01<00:27, 718.85it/s]

  5%|▍         | 1044/21022 [00:01<00:27, 727.19it/s]

  5%|▌         | 1120/21022 [00:01<00:27, 729.81it/s]

  6%|▌         | 1193/21022 [00:01<00:28, 706.15it/s]

  6%|▌         | 1267/21022 [00:01<00:27, 715.12it/s]

  6%|▋         | 1342/21022 [00:01

Running loss: 1.908831

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.169315


Running loss: 0.733652


Running loss: 0.284049


Running loss: 0.239246


Running loss: 0.000454



In [112]:
#check our performance 
result, text = model.eval_model('data/train.json')
print(result)
print(text)



  0%|          | 0/21022 [00:00<?, ?it/s]

  0%|          | 77/21022 [00:00<00:27, 767.36it/s]

  1%|          | 148/21022 [00:00<00:28, 743.16it/s]

  1%|          | 221/21022 [00:00<00:28, 735.64it/s]

  1%|▏         | 302/21022 [00:00<00:27, 754.57it/s]

  2%|▏         | 376/21022 [00:00<00:27, 747.58it/s]

  2%|▏         | 452/21022 [00:00<00:27, 750.67it/s]

  3%|▎         | 534/21022 [00:00<00:26, 768.54it/s]

  3%|▎         | 617/21022 [00:00<00:26, 783.35it/s]

  3%|▎         | 697/21022 [00:00<00:25, 786.51it/s]

  4%|▎         | 779/21022 [00:01<00:25, 795.40it/s]

  4%|▍         | 859/21022 [00:01<00:25, 796.04it/s]

  4%|▍         | 940/21022 [00:01<00:25, 797.75it/s]

  5%|▍         | 1025/21022 [00:01<00:24, 812.66it/s]

  5%|▌         | 1113/21022 [00:01<00:23, 829.69it/s]

  6%|▌         | 1196/21022 [00:01<00:24, 801.09it/s]

  6%|▌         | 1278/21022 [00:01<00:24, 805.60it/s]

  6%|▋         | 1364/21022 [00:01<00:23, 820.41it/s]

  7%|▋         | 1447/21022 [00:0


{'correct': 16563, 'similar': 1333, 'incorrect': 3126, 'eval_loss': -8.456525247028067}
{'correct_text': {'cb774db0d1': 'i`d have responded, if i were going', '088c60f138': 'bullying me', '9642c003ef': 'leave me alone', '358bd9e861': 'sons of ****,', '28b57f3990': 'http://www.dothebouncy.com/smf - some shameless plugging for the best rangers forum on earth', 'e050245fbd': 'both of you', 'fc2cbefa9d': 'wow... u just became cooler.', '16fab9f95b': 'like', '04dd1d2e34': 'lost', 'bbe3cbf620': 'test test from the lg env2', '8a939bfb59': 'uh oh, i am sunburned', '919fa93391': 'sick', 'e48b0b8a23': 'interesting.', '7de057cf40': 'is cleaning the house for her family who is comming later today..', '0c8cc71c46': 'see wat i mean bout foll0w friidays... it`s called lose f0llowers friday... smh', 'e00c6ef376': 'the free fillin` app on my ipod is fun, im addicted', '852edc3769': 'i`m sorry.', 'bdc32ea43c': '.no internet', 'd22e6d40a7': 'power back up not working too', 'd33f811375': 'quite....heaven

In [115]:
print('The training accuracy score is:',result['correct']/len(train))
print(result['similar']/len(train),'results were similar')
print(result['incorrect']/len(train),'results were incorrect')

The training accuracy score is: 0.7878888783179526
0.06340976120254971 results were similar
0.14870136047949767 results were incorrect


In [117]:
#evaluate model on unseen data
result_test, text_test = model.eval_model('data/test2.json')
print(result_test)



  0%|          | 0/3710 [00:00<?, ?it/s]

  2%|▏         | 89/3710 [00:00<00:04, 889.88it/s]

  5%|▍         | 181/3710 [00:00<00:03, 894.90it/s]

  7%|▋         | 254/3710 [00:00<00:04, 836.88it/s]

  9%|▉         | 336/3710 [00:00<00:04, 830.07it/s]

 11%|█▏        | 422/3710 [00:00<00:03, 838.21it/s]

 14%|█▍        | 513/3710 [00:00<00:03, 857.97it/s]

 16%|█▌        | 597/3710 [00:00<00:03, 851.30it/s]

 18%|█▊        | 680/3710 [00:00<00:03, 842.20it/s]

 21%|██        | 765/3710 [00:00<00:03, 842.95it/s]

 23%|██▎       | 850/3710 [00:01<00:03, 844.83it/s]

 25%|██▌       | 941/3710 [00:01<00:03, 862.89it/s]

 28%|██▊       | 1029/3710 [00:01<00:03, 866.85it/s]

 30%|███       | 1115/3710 [00:01<00:03, 824.03it/s]

 32%|███▏      | 1202/3710 [00:01<00:03, 835.42it/s]

 35%|███▍      | 1287/3710 [00:01<00:02, 838.40it/s]

 37%|███▋      | 1375/3710 [00:01<00:02, 849.86it/s]

 39%|███▉      | 1460/3710 [00:01<00:02, 849.31it/s]

 42%|████▏     | 1545/3710 [00:01<00:02, 843.77it/


{'correct': 1590, 'similar': 1422, 'incorrect': 698, 'eval_loss': -8.434787176806351}


In [118]:
print('The testing accuracy score is:',result_test['correct']/len(test2))
print(result_test['similar']/len(test2),'results were similar')
print(result_test['incorrect']/len(test2),'results were incorrect')

The testing accuracy score is: 0.42857142857142855
0.38328840970350403 results were similar
0.1881401617250674 results were incorrect


In [119]:
predictions = model.predict(qa_test)
predictions_df = pd.DataFrame.from_dict(predictions)




  0%|          | 0/3534 [00:00<?, ?it/s]

  2%|▏         | 88/3534 [00:00<00:03, 871.42it/s]

  5%|▍         | 169/3534 [00:00<00:03, 849.25it/s]

  7%|▋         | 249/3534 [00:00<00:03, 833.44it/s]

  9%|▉         | 325/3534 [00:00<00:03, 805.93it/s]

 12%|█▏        | 411/3534 [00:00<00:03, 821.40it/s]

 14%|█▍        | 492/3534 [00:00<00:03, 816.55it/s]

 16%|█▌        | 564/3534 [00:00<00:03, 779.35it/s]

 18%|█▊        | 644/3534 [00:00<00:03, 784.25it/s]

 21%|██        | 732/3534 [00:00<00:03, 807.94it/s]

 23%|██▎       | 821/3534 [00:01<00:03, 829.15it/s]

 26%|██▌       | 906/3534 [00:01<00:03, 832.85it/s]

 28%|██▊       | 994/3534 [00:01<00:03, 845.48it/s]

 31%|███       | 1078/3534 [00:01<00:02, 836.15it/s]

 33%|███▎      | 1161/3534 [00:01<00:02, 830.07it/s]

 35%|███▌      | 1244/3534 [00:01<00:02, 827.53it/s]

 38%|███▊      | 1331/3534 [00:01<00:02, 836.86it/s]

 40%|████      | 1415/3534 [00:01<00:02, 814.20it/s]

 42%|████▏     | 1501/3534 [00:01<00:02, 826.14it/s

In [123]:
predictions_df[1:7]

,id,answer
1,96d74cb729,really exciting
2,eee518ae67,such a shame!
3,01082688c6,happy bday!
4,33987a8ee5,i like it!!
5,726e501993,that`s great!!
6,261932614e,hates
